In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

### MNIST 데이터 준비: MNIST 데이터셋을 다운로드하고 전처리하는 과정입니다.

- 데이터 변환은 transforms.Compose를 사용하여 여러 개의 변환 함수를 순차적으로 적용할 수 있습니다.
- 예를 들어, transforms.ToTensor()는 이미지를 텐서로 변환하고, transforms.Normalize()는 정규화를 수행하게 됩니다.
- 데이터 변환은 모델 학습에 필요한 데이터의 전처리 과정을 수행하는 중요한 단계인 것 같습니다.

In [ ]:
# 1. MNIST 데이터 준비
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

100%|██████████| 9912422/9912422 [00:00<00:00, 82066017.34it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 72929376.17it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 24684889.36it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 14007741.74it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



### (``torch.nn`` 없이) 밑바닥부터 신경망 만들기: torch.nn 모듈을 사용하지 않고, 직접 신경망 모델을 만드는 과정

- 기존에 밑바닥부터 신경망을 구현한 코드를 torch.nn 모듈을 사용하여 코드 리팩토링을 진행했습니다.
- nn.Module을 상속하여 신경망 모델을 정의하고, nn.Linear를 사용하여 레이어를 정의하였습니다.
- 이렇게 모델을 구성함으로써 가독성과 유지보수성이 기존 예시보다 향상되었으며, torch.optim을 사용하여 최적화 알고리즘을 사용했습니다.

In [ ]:
# 2. (``torch.nn`` 없이) 밑바닥부터 신경망 만들기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


- Torch 모듈 사용해 리팩토링 하기: torch.nn.functional, nn.Module, nn.Linear, torch.optim 등의 모듈을 사용하여 코드를 리팩토링하는 과정입니다.

In [ ]:
# 3. Torch 모듈 사용해 리팩토링하기
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

### Dataset을 이용하여 리팩토링하기: torchvision.datasets 모듈을 사용하여 데이터셋을 불러오고 전처리하는 과정입니다.

- torchvision.datasets를 사용하여 MNIST 데이터셋을 불러옴

In [ ]:
# 4. Dataset을 이용하여 리팩토링하기
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

### DataLoader를 사용하여 리팩토링하기: torch.utils.data.DataLoader를 사용하여 데이터를 미니배치로 나누고 셔플하는 과정입니다.

- torch.utils.data.DataLoader를 사용하여 데이터를 미니배치 단위로 로딩했습니다.
- 데이터셋과 데이터로더를 사용하면 데이터를 효율적으로 처리하고 모델에 입력으로 전달할 수 있게 됩니다.
- 여기에 데이터셋을 통해 검증 데이터셋을 분리하고 모델 성능을 평가하는 과정을 추가해 주었습니다.

In [ ]:
# 5. DataLoader를 사용하여 리팩토링하기
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, accuracy

### 검증(validation) 추가하기: 테스트 데이터셋을 사용하여 학습 중에 모델의 성능을 평가하는 과정입니다.

- 모델의 성능을 평가하기 위해 검증(validation) 과정을 추가 했습니다.
- 검증 데이터셋을 사용하여 모델의 손실과 정확도를 계산 했습니다.
- 모델을 평가하는 것은 모델의 일반화 성능을 평가하는 중요한 단계입니다.
- 검증결과를 통해 모델의 학습 진행 상황을 파악 하 수 있게 됩니다.

In [ ]:
# 6. 검증(validation) 추가하기
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 10
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, criterion)
    test_loss, accuracy = test(model, device, test_loader)
    print(f"Epoch: {epoch}, Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch: 1, Test Loss: -6.3647, Accuracy: 90.70%
Epoch: 2, Test Loss: -7.2459, Accuracy: 92.06%
Epoch: 3, Test Loss: -7.5677, Accuracy: 92.67%
Epoch: 4, Test Loss: -7.8709, Accuracy: 93.55%
Epoch: 5, Test Loss: -8.0630, Accuracy: 94.12%
Epoch: 6, Test Loss: -8.1998, Accuracy: 94.60%
Epoch: 7, Test Loss: -8.4240, Accuracy: 94.95%
Epoch: 8, Test Loss: -8.6097, Accuracy: 95.20%
Epoch: 9, Test Loss: -8.7567, Accuracy: 95.36%
Epoch: 10, Test Loss: -8.8986, Accuracy: 95.70%


### fit()와 get_data() 생성하기: fit() 함수를 사용하여 모델을 학습시키는 과정과 get_data() 함수를 사용하여 데이터로더로부터 데이터를 가져오는 과정입니다.

- fit() 함수를 정의하여 학습을 수행하는 과정을 더욱 간결하게 표현했습니다.
- get_data() 함수를 사용하여 데이터로더에서 배치 단위로 데이터를 추출하는 과정을 단순화했습니다.
- 이렇게 함수를 작성하면 코드의 가독성과 재사용성을 향상 시킬 수 있을 것이라 판단했습니다.

In [ ]:
# 7. fit()와 get_data() 생성하기
def fit(model, device, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss / len(train_loader)

def get_data(loader):
    data = []
    for batch_idx, (inputs, labels) in enumerate(loader):
        data.append((inputs, labels))
    return data

train_data = get_data(train_loader)
for epoch in range(1, epochs + 1):
    train_loss = fit(model, device, train_loader, optimizer, criterion)
    print(f"Epoch: {epoch}, Train Loss: {train_loss:.4f}")

Epoch: 1, Train Loss: 0.1324
Epoch: 2, Train Loss: 0.1249
Epoch: 3, Train Loss: 0.1183
Epoch: 4, Train Loss: 0.1124
Epoch: 5, Train Loss: 0.1070
Epoch: 6, Train Loss: 0.1022
Epoch: 7, Train Loss: 0.0978
Epoch: 8, Train Loss: 0.0937
Epoch: 9, Train Loss: 0.0900
Epoch: 10, Train Loss: 0.0865


## 회고
1. 이번 미니 프로젝트를 통해 파이토치를 사용하여 데이터 변환, 모델 구성, 데이터 로딩, 최적화 알고리즘 설정, 검증 과정 등 다양한 기능을 활용하는 방법을 익힐 수 있었습니다. 이를 통해 모델을 학습하고 평가하는 전체적인 프로세스를 이해할 수 있었고, 더욱 실전적인 딥러닝 프로젝트에 대한 이해도를 향상시킬 수 있었던 것 같습니다.

2. CNN 사용하기까지는 결국 이해하지 못했습니다.. 그렇지만 아주 처음부터 차근차근 리팩토링을 해 나가면서, 파이토치의 여러가지 함수들이 계산을 아주 쉽고\빠르게 할 수 있도록 도와주고 있다는 것을 알았습니다. 그래서 파이토치가 텐서플로우보다 훨씬 많이 쓰인다는 것도 알게 되었습니다. 그리고 가장 처음에 하나하나 코드를 작성해 나가면서 순전파와 역전파 과정, 그리고 손실함수랑 최적화 함수를 넣어서 얼마나 손실이 적게 잘 예측할 수 있는지 그 과정을 한땀한땀 같이 할 수 있어서 그나마 조금 이해가 되었던 시간이었습니다.